# Reading raw data from TDS5104 oscilloscope
To run this script, one needs to have TEK VISA library installed. Before opening any resource, make sure the instrument is succesfully identified by TEK VISA Resource Manager.

Import all important modules.

In [1]:
from ipywidgets import interactive, interactive_output, widgets
from tektronix import Waveform, ScopeTest
from matplotlib import pyplot as plt                                                                      
from scipy.signal import savgol_filter
import numpy as np
import pandas as pd
import pyvisa
import os
import pickle
import errno
%matplotlib inline

In [2]:
rlist = []
rm = pyvisa.ResourceManager('')
rlist = rm.list_resources()
data = []

scope = ScopeTest()

w_resources = widgets.Select(options=rlist, description='Available resources')
w_open = widgets.Button(description='Open resource', tooltip='Click to open resource.')
w_chsel = widgets.SelectMultiple(options=[], description='Select channels to be downloaded.')
w_download = widgets.Button(description='Download', tooltip='Click to start the waveform acquisition.')
w_fileName = widgets.Text(value='file.bin', placeholder='Name of the saved file.', description='File:',
                          layout=widgets.Layout(width='250%'))
w_save = widgets.Button(description='Save waveform', tooltip='Click to save waveform file.')
w_reload = widgets.Button(description='Refresh resources', tooltip='Click to refresh the list of available resources.')
w_update = widgets.Button(description='Update channels', tooltip='Click to update list of available waveform sources.')
w_disconnect = widgets.Button(description='Disconnect', tooltip='Click to disconnect from instrument.')

def updateChannels(b):
    chlist = scope.instr.query('DATa:SOUrce:AVAILable?').translate({ord(c): None for c in '\n\r'}).split(',')
    w_chsel.options = chlist
    
def disconnect(b):
    w_chsel.options = []
    scope.instr.close()
    print('Disconnected')

def plotWaveforms(wfms):       
    fig, ax = plt.subplots(1, 1, figsize=(16, 10))
    for i, wfm in enumerate(wfms):
        y = wfm.getRealDiv()
        t = wfm.getTimeBase()

        ax.plot(t * 1e6, y)
        ax.grid()
        
    ax.set_ylim((-5, 5))
    ax.set_yticks(np.linspace(-5, 5, 11))
    ax.axes.yaxis.set_ticklabels([])
    ax.grid(True)


def connect_scope(b):
    print('Opening resource "' + w_resources.value + '"')
    scope.setResource(rm.open_resource(w_resources.value))
    print(scope.instr.query('*IDN?'))
    updateChannels(None)

def printer(sel):
    print('Selected channels: ' + str(sel))
    
def download_data(b):
    w_out.clear_output(wait=True)
    
    try:
        wfms = scope.readWaveforms([ch for ch in w_chsel.value])
        data.clear()
        for wfm in wfms:
            data.append(wfm)
            
    except (visa.VisaIOError):
        print('Channel data not available.')
        return

    with w_out:
        plotWaveforms(data)
        plt.show()
            
def save_file(b):
    fname = w_fileName.value
    if os.path.exists(fname):
        raise FileExistsError('Filename already exists')
    
    if not os.path.exists(os.path.dirname(fname)):
        try:
            os.makedirs(os.path.dirname(fname))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

    with open(fname, 'wb') as file:
        pickle.dump(data, file)
        print('Waveform saved.')

    print('Saved file "' + w_fileName.value + '"')
    
def refresh_resources(b):
    print('Updating available resources...')
    rlist = rm.list_resources()
    w_resources.options = rlist

In [3]:
w_out = widgets.Output()
w_features = widgets.Output()
w_ui = widgets.VBox([widgets.HBox([w_resources, widgets.VBox([w_open, w_reload])]),
                     widgets.HBox([w_chsel, widgets.VBox([w_download, w_update, w_disconnect])]),
                     w_out,
                     widgets.HBox([w_fileName, w_save])])

w_download.on_click(download_data)
w_open.on_click(connect_scope)
w_save.on_click(save_file)
w_reload.on_click(refresh_resources)
w_disconnect.on_click(disconnect)
w_update.on_click(updateChannels)

display(w_ui)

In [4]:
scope.setResource(rm.open_resource('TCPIP0::10.139.2.156::inst0::INSTR'))
print(scope.instr.query('*IDN?'))

TEKTRONIX,MSO66B,C033987,CF:91.1CT FV:1.28.7.8352


In [23]:
chlist = scope.instr.query('DATa:SOUrce:AVAILable?').translate({ord(c): None for c in '\n\r'}).split(',')
chlist

['CH1', 'CH2', 'CH4']

In [22]:
scope.readWaveforms(['CH1', 'CH2'])

[<tektronix.Waveform at 0x25b2056e100>, <tektronix.Waveform at 0x25b2056e190>]

In [4]:
scope.instr.query('ACQuire:SEQuence:CURrent?')

'1'

In [5]:
scope.instr.query('ACQuire:SEQuence:MODe?')

'NUMACQS'

In [10]:
scope.instr.query('ACQuire:STATE?')

'1'

In [28]:
scope.instr.write('ACQuire:STATE RUN')

18

In [10]:
scope.instr.close()

## Testing of enhanced version

In [ ]:
from ipywidgets import interactive, interactive_output
from ipywidgets import widgets as ws
from tektronix import Waveform, ScopeTest
from matplotlib import pyplot as plt
from scipy.signal import savgol_filter
import numpy as np
import pandas as pd
import visa
import os
import pickle
%matplotlib inline